In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
%matplotlib inline


In [5]:
data = pd.read_csv("data.csv", index_col='ID')

FileNotFoundError: [Errno 2] File b'data.csv' does not exist: b'data.csv'

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# Task 1 

In [ ]:
import re
r = re.compile(r"([0-9]+)'([0-9]+)")

def get_inches(el):
    if pd.isnull(el):
            return float('NaN')
    m = r.match(el)
    if m == None:
        return float('NaN')
    else:
        return int(m.group(1))*12 + float(m.group(2))

inches_TO_miter =  39.37  

data['Height_mt'] = data.apply(lambda row: get_inches(row['Height']) / inches_TO_miter, axis=1)

In [ ]:
data['Height_mt'].dropna().hist( bins=30)
 plt.show()

In [ ]:
# Task 2 

In [ ]:
from sklearn.model_selection import train_test_split

data.fillna({'BallControl': data['BallControl'].mean(), 'Dribbling': data['Dribbling'].mean()}, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(data['Dribbling'].values, data['BallControl'].values, train_size=0.8)
X_train = X_train.reshape(-1, 1)
X_test = X_test.reshape(-1, 1)

In [ ]:

from sklearn.linear_model import Ridge

lr = Ridge(alpha=0)
lr.fit(X=X_train, y=y_train)
print(f'w0 = {lr.intercept_}, w1 = {lr.coef_[0]}')

In [ ]:
y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)

In [ ]:
data['predicted_BallControl'] = lr.predict(data['Dribbling'].values.reshape(-1, 1))
data[['Name', 'Dribbling', 'BallControl', 'predicted_BallControl']].head()

In [ ]:
def mse(y_true, y_pred):
    error = np.square(np.subtract(y_true, y_pred)).mean() 
    return error

In [ ]:
from sklearn.metrics import mean_squared_error

assert round(mean_squared_error(y_train, y_pred_train), 9) == round(mse(y_train, y_pred_train), 9)
assert round(mean_squared_error(y_test, y_pred_test), 9) == round(mse(y_test, y_pred_test), 9)

In [ ]:
print(f'Train MSE {mse(y_train, y_pred_train)}, test MSE {mse(y_test, y_pred_test)}')

In [ ]:
# Task 3

In [ ]:
ax_one = data.plot(kind='scatter',x='Dribbling', y='BallControl', color='blue')
data.plot(kind='scatter',x='Dribbling', y='Predicted_BallControl',  color='red', ax=ax_one)
plt.legend(labels=['True', 'Predict'])
plt.title('Dependence of BallControl on Dribbling')
plt.show()

In [ ]:
# Task 4

In [ ]:
def compute_residuals(w, X, y):
    
    
    residuals=[]
    res_temp = 0.0
    for i in range(len(y)):
        for j in range(len(X[i])):
            res_temp=res_temp+X[i][j]*w[j]
        residuals.append(res_temp-y[i])

    return residuals

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from scipy.optimize import least_squares

class LinearRegression(BaseEstimator):
    def __init__(self, fit_intercept=True):
        self.fit_intercept = fit_intercept
    
    def fit(self, X, y):
        """
        fit model weights given input features and target
        :param X: training features, numpy.ndarray: numeric[num_samples, num_features]
        :param y: training target, numpy.ndarray: numeric[num_samples]
        :returns: linear predictor with fitted weights so that train MSE is the lowest possible
        :note: weights: numpy.ndarray: float64[num_features] stored as class field

        """
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        
        # Save train data information. Necessary for following the uniform API
        self.X_ = X
        self.y_ = y
        self.n_features_in_ = X.shape[1]
        
        # Copy arrays and cast them to uniform type
        X_train = X.astype('float64')
        y_train = y.astype('float64')
        
        # Add dummy column of ones to X_train if we want to train an intercept - last component of future weight vector
        if self.fit_intercept:
            X_train = np.column_stack((X_train, np.ones(X_train.shape[0])))
        
        # Your code here.
        # Just follow the suggested steps: create initial weights vector,
        # apply least_squares optimizer passing the parameters described above
        # and finally extract optimized weights.
        # Remember: you need to distinguish coefficients from intercept when fit_intercept=True
        self.w_=[]
        for i in range(len(X_train[0])):
            self.w_.append(0.0)
        if self.fit_intercept:
            self.coef_ = least_squares(compute_residuals,x0=self.w_,args=(X_train,y_train))['x'][:-1]
            self.intercept_ = least_squares(compute_residuals,x0=self.w_,args=(X_train,y_train))['x'][-1]
        else:
            self.coef_ = least_squares(compute_residuals,x0=self.w_,args=(X_train,y_train))['x']
            self.intercept_ = 0
        
        # Return the classifier
        return self
        
    def predict(self, X):
        # Check is fit had been called
        check_is_fitted(self)
        
        # Input validation
        X = check_array(X)
        return X.dot(self.coef_) + self.intercept_

In [ ]:
#Testing area
from sklearn.utils.estimator_checks import check_estimator
from sklearn.linear_model import Ridge

lr = LinearRegression()
ridge = Ridge(alpha=0)
lr_no_intercept = LinearRegression(fit_intercept=False)
ridge_no_intercept = Ridge(alpha=0, fit_intercept=False)

#Check compatibility with Sklearn framework and apply some spesific internal tests
check_estimator(lr)
check_estimator(lr_no_intercept)

#Compare model accuracy with Ridge(0) from Sklearn
data.fillna({'BallControl': data['BallControl'].mean()
             , 'Dribbling': data['Dribbling'].mean()
             , 'Strength': data['Strength'].mean()}, inplace=True)
X_sample, y_sample = data[['Dribbling', 'Strength']], data['BallControl']
lr.fit(X_sample, y_sample)
ridge.fit(X_sample, y_sample)
assert np.allclose(lr.predict(X_sample), ridge.predict(X_sample), rtol=1e-03), "Your model with intercept not accurate enough!"
lr_no_intercept.fit(X_sample, y_sample)
ridge_no_intercept.fit(X_sample, y_sample)
assert np.allclose(lr_no_intercept.predict(X_sample), ridge_no_intercept.predict(X_sample), rtol=1e-03), "Your model without intercept not accurate enough!"

In [ ]:
features = ['BallControl', 'ShortPassing', 'Strength', 'Weight_float', 'Weight_kg']
target = 'Dribbling'
for feat in features:
    data.fillna({feat: data[feat].mean()}, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(data[features].values, data[target].values, train_size=0.8, random_state=2)

In [ ]:
lr = Ridge(0)
lr.fit(X=X_train, y=y_train)

y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)

print(f'Train MSE {mean_squared_error(y_train, y_pred_train)}, test MSE {mean_squared_error(y_test, y_pred_test)}')

In [ ]:

print(f'w_0 = {lr.intercept_}, w_1, w_2, w_3, w_4, w_5 = {lr.coef_}')

In [ ]:

player = data[features + [target]].iloc[0:2]
player['Predicted_dribbling'] = lr.predict(player[features].values)
player.head()

In [ ]:
player['Weight_kg'] = player['Weight_kg'] + [-0.001, 0.001]
player['Predicted_dribbling_with_error'] = lr.predict(player[features].values)
player.head()

In [ ]:
from seaborn import heatmap

heatmap(data[features].corr(method='pearson'), center=0, square=True)
plt.show()

In [ ]:
features = ['BallControl', 'ShortPassing', 'Strength', 'Weight_kg']
X_train, X_test, y_train, y_test = train_test_split(data[features].values, data[target].values, train_size=0.8, random_state=2)

lr = Ridge(alpha=0)
lr.fit(X=X_train, y=y_train)

player['Predicted_dribbling_with_error'] = lr.predict(player[features].values)
player.head()

In [ ]:
#Task5

In [ ]:
Value=data["Value"].str.extract(r'\€([0-9]+?.?[0-9]+)([MK])')
for i in data.index:
    if Value[1][i]=='M':
        data['ValueFloat'][i]=float(Value[0][i])
    else:
        data['ValueFloat'][i]=float(Value[0][i])/1000

In [ ]:
features=['Skill Moves','ShortPassing','HeadingAccuracy','Composure','Acceleration','Volleys','LongShots','Positioning', 'Vision','Reactions', 'Balance']
heatmap(data[features].corr(method='pearson'), center=0, square=True)

In [ ]:

data[features].corr(method='pearson').abs()[data[features].corr(method='pearson')>=0.9].unstack().sort_values(ascending=False).drop_duplicates()

In [ ]:
for feat in features:
    data.fillna({feat: data[feat].mean()}, inplace=True)
data.fillna({'ValueFloat' : data['ValueFloat'].mean()}, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(data[features], data['ValueFloat'], train_size=0.8, random_state=2)

In [ ]:
lr = Ridge(0)
lr.fit(X=X_train, y=y_train)
print(f'w_0 = {lr.intercept_}, w_1, w_2, w_3, w_4 ... w_n = {lr.coef_}')


In [ ]:
y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)
print(f'Train MSE {mean_squared_error(y_train, y_pred_train)}, test MSE {mean_squared_error(y_test, y_pred_test)}')